<a href="https://colab.research.google.com/github/lucasblt/ren-elec-markets/blob/master/exercise3/ex3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pulp as plp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import day_ahead
import balancing_market

## 1.1

In [0]:
g_df = pd.DataFrame({'ID' : ['G1','G2','G3','G4','G5'],
                     'region' : ['E', 'E', 'E', 'E', 'E'],
                     'power_g' : [15, 100, 32, 25, 70],
                     'cost_g' : [75, 15, 0, 42, 10]})

d_df = pd.DataFrame({'ID' : ['D1','D2','D3','D4','D5', 'D6', 'D7'],
                     'region' : ['E', 'E', 'E', 'E', 'E', 'E', 'E'],
                     'power_d' : [35, 23, 12, 38, 43, 16, 57],
                     'cost_d' : [65, 78, 10, 46, 63, 32, 50]})

limit = 0

schedule_g, schedule_d, price = day_ahead.solver(g_df, d_df, limit)

schedule_g['Revenue'] = schedule_g['Scheduled Pg']*schedule_g['Market Price']
schedule_d['Payment'] = schedule_d['Scheduled Pd']*schedule_d['Market Price']

## 1.2

In [0]:
up_df = pd.DataFrame({'ID' : ['G1','G4','G5'],
                     'up_g' : [15, 5, 20],
                     'cost_up' : [80, 50, 35]})

down_df = pd.DataFrame({'ID' : ['G4','G5'],
                     'down_g' : [15, 20],
                     'cost_down' : [15, 5]})

up_df = up_df.sort_values(by=['cost_up'])
up_df['Cumulative'] = up_df['up_g'].cumsum()

down_df = down_df.sort_values(by=['cost_down'], ascending=False)
down_df['Cumulative'] = -down_df['down_g'].cumsum()


In [0]:
down_df['Cumulative'].values

In [0]:
np.flip(down_df['Cumulative'].values)

In [0]:
x = np.concatenate((np.flip(down_df['Cumulative'].values), up_df['Cumulative'].values))
y = np.concatenate((np.flip(down_df['cost_down'].values), up_df['cost_up'].values))

In [0]:
x1 = [-35, -15, 0, 20, 25, 40]
y1 = [ 5, 15, 35, 50, 80, 80]

fig, ax = plt.subplots(figsize=(16, 5))
ax.step(x1,y1,where='post')
ax.axvline(0, linestyle = ':' ,color = 'green')
ax.axvline(22, linestyle = ':' ,color = 'red')
ax.axhline(32, linestyle = ':' ,color = 'green')
ax.axhline(50, linestyle = ':' ,color = 'red')

In [0]:
up_df = pd.DataFrame({'ID' : ['G1','G4','G5'],
                      'region' : ['E', 'E', 'E'],
                      'up_g' : [15, 5, 20],
                      'cost_up' : [80, 50, 35]})

down_df = pd.DataFrame({'ID' : ['G5'],
                        'region' : ['E'],
                        'down_g' : [20],
                        'cost_down' : [5]})
delta = 22

schedule_up, schedule_down, price = balancing_market.solver(up_df, down_df, delta)
schedule_up

,ID,Region,Market Price,Scheduled Pg
0,G1_up,E,50.0,0.0
1,G4_up,E,50.0,2.0
2,G5_up,E,50.0,20.0


In [0]:
up_df = pd.DataFrame({'ID' : ['G1','G4','G5'],
                      'region' : ['E', 'E', 'E'],
                      'up_g' : [15, 5, 20],
                      'cost_up' : [80, 50, 35]})

down_df = pd.DataFrame({'ID' : ['G5'],
                        'region' : ['E'],
                        'down_g' : [20],
                        'cost_down' : [5]})
delta = 10

schedule_up, schedule_down, price = balancing_market.solver(up_df, down_df, delta)
schedule_up

,ID,Region,Market Price,Scheduled Pg
0,G1_up,E,35.0,0.0
1,G4_up,E,35.0,0.0
2,G5_up,E,35.0,10.0
